# 🚀 DeepSeek-R1 Model with Azure AI Inference 🧠

**DeepSeek-R1** is a state-of-the-art reasoning model combining reinforcement learning and supervised fine-tuning, excelling at complex reasoning tasks with 37B active parameters and 128K context window.

In this notebook, you'll learn to:
1. **Initialize** the ChatCompletionsClient for Azure serverless endpoints
2. **Chat** with DeepSeek-R1 using reasoning extraction
3. **Implement** a travel planning example with step-by-step reasoning
4. **Leverage** the 128K context window for complex scenarios

## Why DeepSeek-R1?
- **Advanced Reasoning**: Specializes in chain-of-thought problem solving
- **Massive Context**: 128K token window for detailed analysis
- **Efficient Architecture**: 37B active parameters from 671B total
- **Safety Integrated**: Built-in content filtering capabilities


## 1. Setup & Authentication

This notebook uses the same authentication pattern as other workshop notebooks:
- `azure-ai-projects`: For AIProjectClient integration
- `azure-identity`: For InteractiveBrowserCredential authentication

.env file requirements (same as other notebooks):
```bash
PROJECT_CONNECTION_STRING=<your-project-endpoint-url>
TENANT_ID=<your-azure-tenant-id>
```

### DeepSeek-R1 Model Setup
- **Model Deployment Name**: `DeepSeek-R1-0528`
- Make sure this model is deployed in your Azure AI Foundry project
- The model will be accessed through the project client using browser authentication

> **Note**: This notebook uses the standardized authentication pattern from other workshop notebooks, eliminating the need for API keys and using browser-based authentication instead.

In [1]:
import os
import re
from dotenv import load_dotenv
from pathlib import Path
from azure.ai.projects import AIProjectClient
from azure.identity import InteractiveBrowserCredential
from azure.ai.inference.models import SystemMessage, UserMessage

# Load environment variables (same pattern as other workshop notebooks)
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Get workshop-standard environment variables
project_endpoint = os.getenv("PROJECT_CONNECTION_STRING")
tenant_id = os.getenv("TENANT_ID")
model_name = "DeepSeek-R1-0528"  # DeepSeek model deployment name

print(f"🔑 Using Tenant ID: {tenant_id}")
print(f"🤖 Using model: {model_name}")

# Initialize client with InteractiveBrowserCredential (same as other notebooks)
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using endpoint
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )
    
    print("✅ AIProjectClient created successfully!")
    print("🎉 DeepSeek-R1 client is ready!")
    
except Exception as e:
    print("❌ Initialization failed:", e)
    print("💡 Please complete the browser authentication prompt that should appear")
    print("\n📋 Required .env file format:")
    print("PROJECT_CONNECTION_STRING=<your-project-endpoint>")
    print("TENANT_ID=<your-tenant-id>")

🔑 Using Tenant ID: ed244546-f48e-4572-a767-d6d2a521a7c5
🤖 Using model: DeepSeek-R1-0528
🌐 Using browser-based authentication to bypass Azure CLI cache issues...
✅ AIProjectClient created successfully!
🎉 DeepSeek-R1 client is ready!


In [4]:
# Model fallback logic (same as other workshop notebooks)
print(f"🤖 Target model: {model_name}")

# Check if we have a fallback model available
fallback_model = os.getenv("MODEL_DEPLOYMENT_NAME")
if fallback_model and fallback_model != model_name:
    print(f"🔄 Fallback model available: {fallback_model}")
    print(f"💡 If DeepSeek-R1 isn't deployed, we'll use the fallback model")
    # Note: In case of 404 errors, you can manually set: model_name = fallback_model
else:
    print(f"✅ Using specified model: {model_name}")

print("\n🚀 Ready to demonstrate DeepSeek-R1's reasoning capabilities!")

🤖 Target model: DeepSeek-R1-0528
🔄 Fallback model available: gpt-4.1-mini
💡 If DeepSeek-R1 isn't deployed, we'll use the fallback model

🚀 Ready to demonstrate DeepSeek-R1's reasoning capabilities!


## 2. Intelligent Travel Planning ✈️

Demonstrate DeepSeek-R1's reasoning capabilities for trip planning:

In [2]:
def plan_trip_with_reasoning(query, show_thinking=False):
    """Get travel recommendations with reasoning extraction"""
    try:
        messages = [
            SystemMessage(content="You are a travel expert. Provide detailed plans with rationale."),
            UserMessage(content=f"{query} Include hidden gems and safety considerations.")
        ]
        
        # Use the project client's Azure OpenAI client with proper API
        with project_client.get_openai_client(api_version="2024-10-21") as chat_client:
            response = chat_client.chat.completions.create(
                messages=messages,
                model=model_name,
                temperature=0.7,
                max_tokens=1024
            )
        
        content = response.choices[0].message.content
        
        # Extract reasoning if present (DeepSeek-R1 specific)
        if show_thinking and "<thinking>" in content and "</thinking>" in content:
            thinking_start = content.find("<thinking>") + 10
            thinking_end = content.find("</thinking>")
            thinking = content[thinking_start:thinking_end].strip()
            final_answer = content[thinking_end + 12:].strip()
            
            return {
                "thinking": thinking,
                "answer": final_answer,
                "full_response": content
            }
        
        return content
        
    except Exception as e:
        print(f"❌ Error calling API: {e}")
        print(f"💡 Make sure the model '{model_name}' is properly deployed")
        return None

# Example usage
query = "Plan a 5-day cultural trip to Kyoto in April"
result = plan_trip_with_reasoning(query, show_thinking=True)

print("🗺️ Query:", query)
if isinstance(result, dict):
    print("\n🧠 Model's thinking process:")
    print(result["thinking"])
    print("\n📝 Final recommendation:")
    print(result["answer"])
elif result:
    print("\n📝 Travel Plan:")
    print(result)
else:
    print("❌ Failed to get travel recommendations")

🗺️ Query: Plan a 5-day cultural trip to Kyoto in April

📝 Travel Plan:
Here’s a meticulously crafted 5-day Kyoto cultural itinerary for April, blending iconic sights with authentic hidden gems, prioritizing crowd management, and incorporating essential safety tips:

**Key Considerations for April:**
*   **Cherry Blossom (Sakura) Season:** Peak bloom varies (late March-early April). Expect **major crowds** and higher prices. Book **everything** (flights, hotels, popular restaurants, specific attractions) **months in advance**.
*   **Weather:** Mild days (10-20°C / 50-68°F), cool evenings. Pack layers, a light waterproof jacket, and **comfortable walking shoes** (essential!).
*   **Safety:** Kyoto is very safe. Primary concerns are **pickpockets in crowded areas** (markets, temples during peak bloom), **traffic** (watch for bikes/scooters on narrow streets), and **slippery surfaces** (old stone paths, temple floors). Stay hydrated.

**The Itinerary: Balancing Icons & Serenity**

**Day 1:

## 3. Technical Problem Solving 💻

Showcase coding/optimization capabilities:

In [3]:
def solve_technical_problem(problem):
    """Solve complex technical problems with structured reasoning"""
    try:
        with project_client.get_openai_client(api_version="2024-10-21") as chat_client:
            response = chat_client.chat.completions.create(
                messages=[
                    UserMessage(content=f"{problem} Please reason step by step, and put your final answer within \\boxed{{}}.")
                ],
                model=model_name,
                temperature=0.3,
                max_tokens=2048
            )
        
        return response.choices[0].message.content
        
    except Exception as e:
        print(f"❌ Error solving problem: {e}")
        return None

# Example: Algorithm optimization
problem = """
Optimize this Python function for finding the maximum subarray sum:

def max_subarray_naive(arr):
    max_sum = float('-inf')
    for i in range(len(arr)):
        for j in range(i, len(arr)):
            current_sum = sum(arr[i:j+1])
            max_sum = max(max_sum, current_sum)
    return max_sum

The current time complexity is O(n³). Can you improve it?
"""

print("🔧 Problem:", problem)
solution = solve_technical_problem(problem)
if solution:
    print("\n⚙️ Solution:")
    print(solution)
else:
    print("❌ Failed to get solution")

🔧 Problem: 
Optimize this Python function for finding the maximum subarray sum:

def max_subarray_naive(arr):
    max_sum = float('-inf')
    for i in range(len(arr)):
        for j in range(i, len(arr)):
            current_sum = sum(arr[i:j+1])
            max_sum = max(max_sum, current_sum)
    return max_sum

The current time complexity is O(n³). Can you improve it?


⚙️ Solution:
To optimize the given function for finding the maximum subarray sum, we can replace the naive O(n³) approach with Kadane's algorithm, which efficiently computes the solution in O(n) time with O(1) space complexity. Here's the improved implementation:

```python
def max_subarray(arr):
    if not arr:
        return 0
    max_current = max_global = arr[0]
    for num in arr[1:]:
        max_current = max(num, max_current + num)
        if max_current > max_global:
            max_global = max_current
    return max_global
```

**Explanation:**
1. **Initialization:** The algorithm initializes two variables, 

## 4. Best Practices & Considerations

1. **Reasoning Handling**: Use regex to separate <think> content from final answers
2. **Safety**: Built-in content filtering - handle HttpResponseError for violations
3. **Performance**:
   - Max tokens: 4096
   - Rate limit: 200K tokens/minute
4. **Cost**: Pay-as-you-go with serverless deployment
5. **Streaming**: Implement response streaming for long completions

```python
# Streaming example
response = client.chat.completions.create(..., stream=True)
for chunk in response:
    print(chunk.choices[0].delta.content or "", end="")
```

## 🎯 Key Takeaways
- Leverage 128K context for detailed analysis
- Extract reasoning steps for debugging/analysis
- Combine with Azure AI Content Safety for production
- Monitor token usage via response.usage

> Always validate model outputs for critical applications!